# stko and stk

by: Andrew Tarzia

### Update 25/02/24

Installation of py3Dmol using `pip install py3Dmol` is required.

# Imports 

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
import subprocess
import time
import stk
import stko
import spindry as spd
%matplotlib inline

In [ ]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

# Working

In [ ]:
bb1 = stk.BuildingBlock('NCCN', [stk.PrimaryAminoFactory()])
bb2 = stk.BuildingBlock(
    smiles='O=CC(C=O)C=O',
    functional_groups=[stk.AldehydeFactory()],
)
cage1 = stk.ConstructedMolecule(
    topology_graph=stk.cage.FourPlusSix((bb1, bb2)),
)

In [ ]:
show_stk_mol(cage1)

In [ ]:
broken_bonds_by_id = []
disconnectors = []
for bi in cage1.get_bond_infos():
    if bi.get_building_block() is None:
        a1id = bi.get_bond().get_atom1().get_id()
        a2id = bi.get_bond().get_atom2().get_id()
        broken_bonds_by_id.append(sorted((a1id, a2id)))
        disconnectors.extend((a1id, a2id))

In [ ]:
print(broken_bonds_by_id)
print(disconnectors)
print('--')

In [ ]:
new_topology_graph = stko.TopologyExtractor()
tg_info = new_topology_graph.extract_topology(
    molecule=cage1,
    broken_bonds_by_id=broken_bonds_by_id,
    disconnectors=set(disconnectors),
)

In [ ]:
import os
os.mkdir('tg_output')

In [ ]:
print(tg_info.get_vertex_positions())
print(tg_info.get_connectivities())
print(tg_info.get_edge_pairs())

In [ ]:
cage1.write('tg_output/tg_cage.mol')
tg_info.write('tg_output/tg_info.pdb')